# Example: Measuring GFP positive neuron proportion

This notebook demonstrates how we can classify nuclei as either positive or negative for two differet markers, and then calculate the overlap between these.

Specifically, this pipeline was used to calculate the proportion of neurons that expressed the gfp fusion protein indicating successfull knockout of the TDG gene.

## Import dependencies

The following libraries are used for the data processing, clustering, statistical calculations and visualization

In [ ]:
import pandas as pd
import napari
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import MultiComparison
from sklearn.cluster import KMeans
import numpy as np
import sys
sys.path.insert(1, '/Users/jonas/Documents/nucleus_analysis/nuclei')
from utils import readImage
pd.set_option('display.width', 2000)
import warnings
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows with MKL")

## Load .csv file

The .csv file is loaded, and a Pandas dataframe is created from it.
We inspect the structure of the dataframe by printing it.

In [ ]:
nucleus_df = pd.read_csv('csv/nuclei_liv_test.csv')
print(nucleus_df)

## Classifying Neurons

Firstly, we wish to identify the neurons of the images. This is done by clustering based on NeuN fluorescence.

A k-means clustering is conducted. n_clusters indicates the number of clusters. In this case, this is two: Neurons and non-neurons.



In [ ]:
#channel_key inticates which column we should cluster by.
channel_key = 'Ch1Intensity'
nucleus_df['Cluster_Label'] = np.nan
num_clusters = 2

for name in nucleus_df['ImageName'].unique():
    df = nucleus_df[nucleus_df['ImageName'] == name]
    intensity_values = df[channel_key].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init="auto")
    clusters = kmeans.fit_predict(intensity_values)
    sorted_clusters = np.argsort(kmeans.cluster_centers_.flatten())
    cluster_labels = sorted_clusters[clusters] 
    nucleus_df.loc[df.index, 'Cluster_Label'] = cluster_labels

nucleus_df.loc[nucleus_df['Cluster_Label'] == 1.0, 'CellType'] = 'Neuron'
nucleus_df.drop(columns=['Cluster_Label'], inplace=True)
print(nucleus_df.head())

### Verify classification

In order to inspect whether or not our classification was successfull we may both:
- Display positive nuclei in the image

In [ ]:
img_name = '170123_A1_363_NeuN-GFP_S1_MEC.czi' # 'Enter name of image as it appears in nucleus_df'
img_path = 'D:/Users\Jonas/nuclei\imagesAndMasks\liv\mec/170123_A1_363_NeuN-GFP_S1_MEC.czi' # 'Path to same image'

neuron_df = nucleus_df[nucleus_df['CellType'] == 'Neuron']
img_df = neuron_df[neuron_df['ImageName'] == img_name]

centroids = img_df['Centroid'].apply(eval)

# Convert centroids to a NumPy array
centroids_array = np.array(centroids.tolist())  # Transpose the array to get the desired shape


from skimage import io
import napari
img= readImage(img_path)

viewer = napari.view_image(img, scale =([0.9278, 0.3459, 0.3459]), channel_axis=3 )
viewer.add_points(centroids_array, size=10, symbol='cross', edge_color='blue', face_color='blue', name='Nuclei Centroids', scale=([0.9278, 0.3459, 0.3459]))
napari.run()

- Combination of violinplot and boxplot

In [ ]:
channel_key = 'Ch1Intensity'

for name in nucleus_df['ImageName'].unique():
    neuron_intensities = []
    undefined_intensities = []
    df = nucleus_df[nucleus_df['ImageName'] == name]
    neuron_intensities.extend(df[df['CellType'] == 'Neuron'][channel_key].tolist())
    # Filter 'Ch1Intensity' values where 'CellType' == 'Undefined'
    undefined_intensities.extend(df[df['CellType'] == 'Undefined'][channel_key].tolist())
    shared_values = set(neuron_intensities) & set(undefined_intensities)
    if shared_values:
        print(f"Shared values found: {shared_values}")
    fig, ax = plt.subplots()
    sns.violinplot(data=df, x='ImageName', y=channel_key, zorder=1, ax=ax)
    ax.boxplot(neuron_intensities, positions=[0], widths=0.2, patch_artist=True,
            boxprops=dict(facecolor="seagreen", zorder=2, alpha=.7),
            medianprops=dict(color="black"),
            whiskerprops=dict(color="darkgreen"),
            capprops=dict(color="darkgreen"))

    ax.boxplot(undefined_intensities, positions=[0], widths=0.2, patch_artist=True,
            boxprops=dict(facecolor="lightcoral", zorder=2, alpha=.7),
            medianprops=dict(color="black"),
            whiskerprops=dict(color="firebrick"),
            capprops=dict(color="firebrick"))
    plt.xlabel(name)
    plt.xticks([0],"")
    plt.show()

## Classify GFP positive 

In [ ]:
channel_key = 'Ch3Intensity'
nucleus_df['Cluster_Label'] = np.nan
nucleus_df['GFP_positive'] = np.nan


for name in nucleus_df['ImageName'].unique():
    df = nucleus_df[nucleus_df['ImageName'] == name]
    intensity_values = df[channel_key].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=42, n_init="auto")
    clusters = kmeans.fit_predict(intensity_values)
    sorted_clusters = np.argsort(kmeans.cluster_centers_.flatten())
    cluster_labels = sorted_clusters[clusters]  # Map cluster indices to sorted clusters
    nucleus_df.loc[df.index, 'Cluster_Label'] = cluster_labels

nucleus_df.loc[nucleus_df['Cluster_Label'] == 1.0, 'GFP_positive'] = True
nucleus_df.drop(columns=['Cluster_Label'], inplace=True)
print(nucleus_df.head())

In [ ]:
result = nucleus_df.groupby('ImageName').agg(
    Condition=('Condition', 'first'),
    TotalNuclei=('Label', 'count'),
    NeuronNuclei=('CellType', lambda x: (x == 'Neuron').sum()),
     GFPPositiveNuclei=('GFP_positive', lambda x: x.sum()),
     NeuronAndGFPPositive=('GFP_positive', lambda x: ((x == True) & (nucleus_df['CellType'] == 'Neuron')).sum())
)
result['ProportionNeuronAndGFP'] = result['NeuronAndGFPPositive'] / result['NeuronNuclei']
print(result)

## Visualize results

In [ ]:
sns.boxplot(data=result, x='Condition', y='ProportionNeuronAndGFP', showfliers=False,order=['CA1','CA3','DG', 'MEC'], palette="Set3")
sns.stripplot(data=result, x='Condition', y='ProportionNeuronAndGFP', order=['CA1','CA3','DG', 'MEC'], alpha=0.7)
plt.xlabel("Region")
plt.ylabel("Proportion GFP+ Neurons")
#plt.savefig('gfpNeurons.pdf')
plt.show()